# for testing (all the tracks are used)

In [1]:
import pandas as pd
import numpy as np



In [2]:
# kmean100_df = pd.read_csv('../data/interim/all_data/mbKMeans100clusters.csv', usecols=['track_id','clus'])
# kmean100_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)



In [6]:
from dask.dataframe import from_pandas
from dask.dataframe.reshape import pivot_table
import dask.array as da

from timeit import default_timer as timer #to see how long the computation will take

def cal_svd(filename, kmean_df):
#     start = timer()
    df = pd.read_csv(filename, usecols = ['session_id','skip_2','track_id_clean','session_length','session_position']).merge(kmean_df)
    df = df.astype({'session_id':'category', 'track_id_clean':'category', 'clus': 'category', 'session_length':'int32', 'session_position':'int32'})
#     df = df.loc[df['session_position']<(df['session_length']/2)]
    df.drop(columns = ['track_id_clean','session_length','session_position'], inplace = True)
    df['ListenYes'] = (df['skip_2'] == False)*1
    df['ListenYes'].replace(0, -1, inplace = True)
    

    df2 = df.groupby(['session_id', 'clus']).agg({'ListenYes':['sum']}).reset_index()
    df2.columns = df2.columns.droplevel(level = 1) # take out the unwanted level
#     make a dask dataframe
    dd = from_pandas(df2, npartitions=1)
    del df, df2
    DfMatrix = pivot_table(dd, values='ListenYes', index='session_id', columns='clus')
    v = da.linalg.svd(DfMatrix.to_dask_array())[2]
    pd.DataFrame(v.compute()).to_csv('../models/SVD/all_tracks/k'+str(kmean_df['clus'].nunique())+'_v_'+filename[25:])


In [7]:
import glob
kmean100_df = pd.read_csv('../data/interim/all_data/mbKMeans100clusters.csv', usecols=['track_id','clus'])
kmean100_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)
for n in range(9):
    file_list = glob.glob('../data/raw/training_set/log_'+str(n)+'*.csv')
    for filename in file_list:
        v = cal_svd(filename, kmean100_df)

In [8]:
import glob
kmean300_df = pd.read_csv('../data/interim/all_data/mbKMeans300clusters.csv', usecols=['track_id','clus'])
kmean300_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)
for n in range(9):
    file_list = glob.glob('../data/raw/training_set/log_'+str(n)+'*.csv')
    for filename in file_list:
        v = cal_svd(filename, kmean300_df)

In [17]:
dd = from_pandas(v, npartitions=10)
dd

,session_id,clus,ListenYes
npartitions=10,,,
0,category[known],category[known],int64
5164740,...,...,...
...,...,...,...
46482660,...,...,...
51647399,...,...,...


In [18]:
DfMatrix = pivot_table(dd, values='ListenYes', index='session_id', columns='clus')

In [23]:
DfMatrix.compute()

clus,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
session_id,,,,,,,,,,,,,,,,,,,,,
7_00007784-914d-4fa0-a1ef-cc4ee90c895b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7_0000b75f-4790-435a-b0d9-928ac182569d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7_0000d029-2d6d-49b3-a8c0-956306f33538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7_0002d439-3fad-4b00-9c2d-242b13337be5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7_0003d9ba-a650-41d5-b70a-e12337256ca5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7_fffec6f7-9be0-4296-b034-86130a91da2a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7_fffed5c4-67d3-4b59-b9e8-6a6c67414380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7_ffff6fa5-e63c-4e37-8cb0-dfc8c9fea588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
v = da.linalg.svd(DfMatrix.to_dask_array())[2]

In [21]:
v

dask.array<mul, shape=(nan, 300), dtype=float64, chunksize=(nan, 300), chunktype=numpy.ndarray>

In [22]:
v.shape

(nan, 300)